In [1]:
#Imports
import os
import numpy as np
import pandas as pd
import requests
import json
from datetime import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

In [2]:
#Get API NEWS 12-01 - present
December = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=news,urls&limit=10000&type=influential&start=1606780800&"
DecemberRequest = requests.get(December)
DecemberData = DecemberRequest.json()

#Get API NEWS 10-01 - 11-30
October = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=news,urls&limit=10000&type=influential&start=1601510400&end=1606694400"
OctoberRequest = requests.get(October)
Octoberdata = OctoberRequest.json()

#Get API NEWS 8-01 - 9-30
August = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=news,urls&limit=10000&type=influential&start=1596240001&end=1601424001"
AugustRequest = requests.get(August)
Augustdata = AugustRequest.json()

#Get API NEWS 6-01 - 7-31
June = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=news,urls&limit=10000&type=influential&start=1590969601&end=1596153601"
JuneRequest = requests.get(June)
Junedata = JuneRequest.json()

#Get API NEWS 4-01 - 5-31
April = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=news,urls&limit=10000&type=influential&start=1585699201&end=1590883201"
AprilRequest = requests.get(April)
Aprildata = AprilRequest.json()

#Get API NEWS 2-01 - 3-31
February = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=news,urls&limit=10000&type=influential&start=1580515201&end=1585612801"
FebruaryRequest = requests.get(February)
Februarydata = FebruaryRequest.json()

#Get API NEWS 12-01 - 1-31 (2019)
Dec = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=news,urls&limit=10000&type=influential&start=1575158401&end=1580428801"
DecRequest = requests.get(Dec)
Decdata = DecRequest.json()

#Get API NEWS 10-01 - 11-30 (2019)
Oct = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=news,urls&limit=10000&type=influential&start=1569888001&end=1575072001"
OctRequest = requests.get(Oct)
Octdata = OctRequest.json()

#Get API NEWS 8-01 - 9-30 (2019)
Aug = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=news,urls&limit=10000&type=influential&start=1564617601&end=1569801601"
AugRequest = requests.get(Aug)
Augdata = AugRequest.json()

#Get API NEWS 6-01 - 7-31 (2019)
Jun = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=news,urls&limit=10000&type=influential&start=1559347201&end=1564531201"
JunRequest = requests.get(Jun)
Jundata = JunRequest.json()

#Get API NEWS 4-01 - 5-31 (2019)
Apr = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=news,urls&limit=10000&type=influential&start=1554076801&end=1559260801"
AprRequest = requests.get(Apr)
Aprdata = AprRequest.json()

#Get API NEWS 2-01 - 3-31 (2019)
Feb = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=news,urls&limit=10000&type=influential&start=1548979201&end=1553990401"
FebRequest = requests.get(Feb)
Febdata = FebRequest.json()

#Get API NEWS 1-01 - 1-31 (2019)
Jan = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=news,urls&limit=10000&type=influential&start=1546300801&end=1548892801"
JanRequest = requests.get(Jan)
Jandata = JanRequest.json()

In [3]:
def get_sentiment_score(data_json):
    sentiments = []

    for article in data_json['data']:
        try:
            text = article["description"]
            date = article["time"]
            symbol = article["symbol"]
            sentiment = analyzer.polarity_scores(text)
            compound = sentiment["compound"]
            pos = sentiment["pos"]
            neu = sentiment["neu"]
            neg = sentiment["neg"]
            sentiments.append({
                "Date": date,
                "Ticker": symbol,
                "Compound": compound,
                "Positive": pos,
                "Negative": neg,
                "Neutral": neu,
                #"Text": text 
            })
            
        except AttributeError:
            pass 

    # Create DataFrame
    df = pd.DataFrame(sentiments)
    return df


In [4]:
Dec20 = get_sentiment_score(DecemberData)
Oct20 = get_sentiment_score(Octoberdata)
Aug20 = get_sentiment_score(Augustdata)
Jun20 = get_sentiment_score(Junedata)
Apr20 = get_sentiment_score(Aprildata)
Feb20 = get_sentiment_score(Februarydata)
Dec19 = get_sentiment_score(Decdata)
Oct19 = get_sentiment_score(Octdata)
Aug19 = get_sentiment_score(Augdata)
Jun19 = get_sentiment_score(Jundata)
Apr19 = get_sentiment_score(Aprdata)
Feb19 = get_sentiment_score(Febdata)
Jan19 = get_sentiment_score(Jandata)

CoinsNewsDF = pd.concat([Dec20,Oct20, Aug20, Jun20, Apr20, Feb20, Dec19, Oct19, Aug19, Jun19, Apr19, Feb19, Jan19], axis="rows", ignore_index=True).reset_index(drop=True)
CoinsNewsDF = CoinsNewsDF.drop_duplicates()
CoinsNewsDF.tail()


,Date,Ticker,Compound,Positive,Negative,Neutral
52331,1569592648,DOGE,-0.7096,0.000,0.211,0.789
52332,1515165207,DOGE,0.3400,0.156,0.000,0.844
52333,1569750874,DOGE,0.6705,0.180,0.000,0.820
52334,1569753308,DOGE,0.0000,0.000,0.000,1.000
52335,1569777039,DOGE,0.0000,0.000,0.000,1.000


In [8]:
#Get API SM 12-01 - present
DecemberSM = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=twitter,reddit&limit=500&type=influential&start=1606780800"
DecemberRequestSM = requests.get(DecemberSM)
DecemberDataSM = DecemberRequestSM.json()

#Get API SM 10-01 - 11-30
OctoberSM = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=twitter,reddit&limit=500&type=influential&start=1601510400&end=1606694400"
OctoberRequestSM = requests.get(OctoberSM)
OctoberdataSM = OctoberRequestSM.json()

#Get API SM 8-01 - 9-30
AugustSM = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=twitter,reddit&limit=500&type=influential&start=1596240001&end=1601424001"
AugustRequestSM = requests.get(AugustSM)
AugustdataSM = AugustRequestSM.json()

#Get API SM 6-01 - 7-31
JuneSM = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=twitter,reddit&limit=500&type=influential&start=1590969601&end=1596153601"
JuneRequestSM = requests.get(JuneSM)
JunedataSM = JuneRequestSM.json()

#Get API SM 4-01 - 5-31
AprilSM = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=twitter,reddit&limit=500&type=influential&start=1585699201&end=1590883201"
AprilRequestSM = requests.get(AprilSM)
AprildataSM = AprilRequestSM.json()

#Get API SM 2-01 - 3-31
FebruarySM = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=twitter,reddit&limit=500&type=influential&start=1580515201&end=1585612801"
FebruaryRequestSM = requests.get(FebruarySM)
FebruarydataSM = FebruaryRequestSM.json()

#Get API SM 12-01 - 1-31 (2019)
DecSM = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=twitter,reddit&limit=500&type=influential&start=1575158401&end=1580428801"
DecRequestSM = requests.get(DecSM)
DecdataSM = DecRequestSM.json()

#Get API SM 10-01 - 11-30 (2019)
OctSM = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=twitter,reddit&limit=500&type=influential&start=1569888001&end=1575072001"
OctRequestSM = requests.get(OctSM)
OctdataSM = OctRequestSM.json()

#Get API SM 8-01 - 9-30 (2019)
AugSM = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=twitter,reddit&limit=500&type=influential&start=1564617601&end=1569801601"
AugRequestSM = requests.get(AugSM)
AugdataSM = AugRequestSM.json()

#Get API SM 6-01 - 7-31 (2019)
JunSM = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=twitter,reddit&limit=500&type=influential&start=1559347201&end=1564531201"
JunRequestSM = requests.get(JunSM)
JundataSM = JunRequestSM.json()

#Get API SM 4-01 - 5-31 (2019)
AprSM = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=twitter,reddit&limit=500&type=influential&start=1554076801&end=1559260801"
AprRequestSM = requests.get(AprSM)
AprdataSM = AprRequestSM.json()

#Get API SM 2-01 - 3-31 (2019)
FebSM = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=twitter,reddit&limit=500&type=influential&start=1548979201&end=1553990401"
FebRequestSM = requests.get(FebSM)
FebdataSM = FebRequestSM.json()

#Get API SM 1-01 - 1-31 (2019)
JanSM = "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,xmr,xlm,doge,trx,nano,dot&sources=twitter,reddit&limit=500&type=influential&start=1546300801&end=1548892801"
JanRequestSM = requests.get(JanSM)
JandataSM = JanRequestSM.json()

In [9]:
def get_smscore(smdata_json):
    sentiment = []

    for article in smdata_json["data"]:
        try:
            text = article["body"]
            date = article["time"]
            symbol = article["symbol"]
            sentiments = analyzer.polarity_scores(text)
            compound = sentiments["compound"]
            pos = sentiments["pos"]
            neu = sentiments["neu"]
            neg = sentiments["neg"]
            sentiment.append({
                "Date": date,
                "Ticker": symbol,
                "Compound": compound,
                "Positive": pos,
                "Negative": neg,
                "Neutral": neu,
                #"Text": text 
            })
            #print(sentiment)

        except AttributeError:
            pass 

    # Create DataFrame
    df1 = pd.DataFrame(sentiment)
    return df1


In [10]:
Dec20SM = get_smscore(OctoberdataSM)
Oct20SM = get_smscore(OctoberdataSM)
Aug20SM = get_smscore(AugustdataSM)
Jun20SM = get_smscore(JunedataSM)
Apr20SM = get_smscore(AprildataSM)
Feb20SM = get_smscore(FebruarydataSM)
Dec19SM = get_smscore(DecdataSM)
Oct19SM = get_smscore(OctdataSM)
Aug19SM = get_smscore(AugdataSM)
Jun19SM = get_smscore(JundataSM)
Apr19SM = get_smscore(AprdataSM)
Feb19SM = get_smscore(FebdataSM)
Jan19SM = get_smscore(JandataSM)

CoinsSMDF = pd.concat([Dec20SM,Oct20SM, Aug20SM, Jun20SM, Apr20SM, Feb20SM, Dec19SM, Oct19SM, Aug19SM, Jun19SM, Apr19SM, Feb19SM, Jan19SM], axis="rows", ignore_index=True).reset_index(drop=True)
CoinsNewsDF = CoinsNewsDF.drop_duplicates()
CoinsSMDF.tail()

,Date,Ticker,Compound,Positive,Negative,Neutral
6403,1547826944,TRX,0.9835,0.116,0.009,0.875
6404,1546973954,LTC,0.0000,0.000,0.000,0.000
6405,1547749681,TRX,0.0000,0.000,0.000,0.000
6406,1546697785,XMR,0.9923,0.066,0.044,0.890
6407,1547581647,TRX,0.0000,0.000,0.000,0.000


In [ ]:
#Prep dataset

Coins_Sentiment = pd.concat([CoinsNewsDF,CoinsSMDF], axis="rows", ignore_index=True)
Coins_Sentiment['Date'] = pd.to_datetime(Coins_Sentiment['Date'],unit='s')

#Coins_Sentiment['Date'] = pd.to_datetime(Coins_Sentiment['Date']).dt.date

Coins_Sentiment = Coins_Sentiment.groupby(['Date','Ticker'],as_index=False).mean()
Coins_Sentiment = Coins_Sentiment.sort_values(by=['Date'], ascending=False,ignore_index=True)
Coins_Sentiment.set_index('Date', inplace=True, drop=True)

#BTC_Sentiment.reset_index(drop=True, inplace=True)
#BTC_Sentiment.reset_index(inplace=True)
#BTC_Sentiment.columns = ['Date', 'Ticker', 'Compound', 'Positive', 'Negative', 'Neutral']

Coins_Sentiment.head()

In [ ]:
##Create Sentiment Signal column

# create a list of our conditions
conditions = [
    (Coins_Sentiment['Compound'] < 0),
    (Coins_Sentiment['Compound'] == 0),
    (Coins_Sentiment['Compound'] > 0)
    ]

# create a list of the values we want to assign for each condition
values = ['-1', '0', '1']

# create a new column and use np.select to assign values to it using our lists as arguments
Coins_Sentiment['Sentiment Signal'] = np.select(conditions, values)

# display updated DataFrame
Coins_Sentiment.head()

In [ ]:
#Slice Bitcoin from DF
BitcoinDF = Coins_Sentiment.loc[Coins_Sentiment['Ticker'] == 'BTC']

#Resample data
BitcoinDF = BitcoinDF.resample('D').bfill()
BitcoinDF = BitcoinDF.sort_values(by=['Date'], ascending=False)

#BitcoinDF.tail()
BitcoinDF.to_csv("../libs/data/BitcoinSentimentSignals.csv")

In [ ]:
#Slice Litecoin from DF
LitecoinDF = Coins_Sentiment.loc[Coins_Sentiment['Ticker'] == 'LTC']

#Resample data
LitecoinDF = LitecoinDF.resample('D').bfill()
LitecoinDF = LitecoinDF.sort_values(by=['Date'], ascending=False)

#LitecoinDF.head()
LitecoinDF.to_csv("../libs/data/LitecoinSentimentSignals.csv")

In [ ]:
#Slice Polkadot from DF
PolkadotDF = Coins_Sentiment.loc[Coins_Sentiment['Ticker'] == 'DOT']

#Resample data
PolkadotDF = PolkadotDF.resample('D').bfill()
PolkadotDF = PolkadotDF.sort_values(by=['Date'], ascending=False)

#PolkadotDF.head(50)
PolkadotDF.to_csv("../libs/data/PolkadotSentimentSignals.csv")

In [ ]:
#Slice Monero from DF
MoneroDF = Coins_Sentiment.loc[Coins_Sentiment['Ticker'] == 'XMR']

#Resample data
MoneroDF = MoneroDF.resample('D').bfill()
MoneroDF = MoneroDF.sort_values(by=['Date'], ascending=False)

#MoneroDF.head()
MoneroDF.to_csv("../libs/data/MoneroSentimentSignals.csv")

In [ ]:
#Slice Stellar Lumens from DF
StellarLumensDF = Coins_Sentiment.loc[Coins_Sentiment['Ticker'] == 'XLM']

#Resample data
StellarLumensDF = StellarLumensDF.resample('D').bfill()
StellarLumensDF = StellarLumensDF.sort_values(by=['Date'], ascending=False)

#StellarLumensDF.head()
StellarLumensDF.to_csv("../libs/data/StellarLumensSentimentSignals.csv")

In [ ]:
#Slice Dogecoin from DF
DogecoinDF = Coins_Sentiment.loc[Coins_Sentiment['Ticker'] == 'DOGE']

#Resample data
DogecoinDF = DogecoinDF.resample('D').bfill()
DogecoinDF = DogecoinDF.sort_values(by=['Date'], ascending=False)

#StellarLumensDF.head()
DogecoinDF.to_csv("../libs/data/DogecoinSentimentSignals.csv")

In [ ]:
#Slice Nano from DF
NanoDF = Coins_Sentiment.loc[Coins_Sentiment['Ticker'] == 'NANO']

#Resample data
NanoDF = NanoDF.resample('D').bfill()
NanoDF = NanoDF.sort_values(by=['Date'], ascending=False)

#StellarLumensDF.head()
NanoDF.to_csv("../libs/data/NanoSentimentSignals.csv")

In [ ]:
#Slice Tron from DF
TronDF = Coins_Sentiment.loc[Coins_Sentiment['Ticker'] == 'TRX']

#Resample data
TronDF = TronDF.resample('D').bfill()
TronDF = TronDF.sort_values(by=['Date'], ascending=False)

#StellarLumensDF.head()
TronDF.to_csv("../libs/data/TronSentimentSignals.csv")